In [19]:
import json
import requests
from pprint import pprint
import random
import getpass

In [11]:
OAUTH = getpass.getpass('RW Token_ID:')

RW Token_ID: ·······························································································································································································································································································································································································


## Updating Widget

In [254]:
""" Copy the old one and change desired values (in this case attributes.description)
    NOTE that you cannot update id, or type, only attributes!
"""

payload = {

    
}

In [335]:
""" Copy the old one and change desired values (in this case attributes.description)
    NOTE that you cannot update id, or type, only attributes!
"""

payload = {

            "widgetConfig": {
                "data": [
                    {
                        "url": "https://wri-rw.carto.com/api/v2/sql?q=SELECT gid_0 as iso, name_0 as country, name_1 as province, indicator_name, weight, score, score_ranked FROM aqueduct_results_v01_province_v03 WHERE indicator_name = {{indicator}} and gid_0 = {{iso}} and weight in ('Tot', 'Irr', 'Dom', 'Ind', 'Pop') ORDER BY score_ranked "
                    }
                ],
                "params_config": [
                    {
                        "key": "indicator",
                        "required": True
                    },
                    {
                        "key": "iso",
                        "required": True
                    }
                ]
            },
    
}

In [336]:
#Update widget
dataset_id = 'e0adfbfb-441d-4ec5-9e4c-7d051d0f7b22'
widget_id = '4b3d6e89-869c-45b6-88e3-28735d2b60fe'

url = f'https://staging-api.globalforestwatch.org/v1/dataset/{dataset_id}/widget/{widget_id}'

headers = {'Authorization': 'Bearer ' + OAUTH, 'Content-Type': 'application/json'}

r = requests.patch(url, data=json.dumps(payload), headers=headers)

pprint(r.json())

{'data': {'attributes': {'application': ['aqueduct'],
                         'authors': '',
                         'createdAt': '2019-07-10T11:04:25.902Z',
                         'dataset': 'e0adfbfb-441d-4ec5-9e4c-7d051d0f7b22',
                         'default': False,
                         'defaultEditableWidget': False,
                         'description': '',
                         'env': 'production',
                         'freeze': False,
                         'name': 'Province rankings',
                         'protected': False,
                         'published': True,
                         'slug': 'Province-rankings',
                         'source': '',
                         'template': False,
                         'thumbnailUrl': 'http://wri-api-backups.s3.amazonaws.com/resourcewatch/staging/thumbnails/4b3d6e89-869c-45b6-88e3-28735d2b60fe-1562756665947.png',
                         'updatedAt': '2019-07-11T14:35:00.467Z',
              